## Metrics Classes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Proofpoint

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Proofpoint


In [ ]:
!pip install mlflow --quiet

     |████████████████████████████████| 17.0 MB 30.9 MB/s 
     |████████████████████████████████| 209 kB 92.8 MB/s 
     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 182 kB 77.8 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 147 kB 84.6 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 140 kB 102.3 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [ ]:
import torch
from torchsummary import summary
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import mlflow
import mlflow.sklearn
import sys
import os


In [ ]:
"""
This script contains functions and classes to compute churn metrics
UNTESTED
"""

import numpy as np
import torch
from typing import Union

class ChurnMetric:
    """
    Super class for metrics. 
    """
    def __init__(self, tensor_type="numpy") -> None:
        tensor_types = {"numpy": np.ndarray, "torch": torch.Tensor}
        if tensor_type not in tensor_types:
            raise NotImplementedError("Unknown object type")
        self.tensor_type = tensor_types[tensor_type]
    
    def call_sanitize_inputs(self, **preds):
        """
        preds can be true labels as well
        Shape of tensors need to be the same.
        Tensor dim len must not be >2
        """
        for p in preds:
            if not isinstance(preds[p], self.tensor_type):
                raise TypeError(f"{p} is not an instance of {str(self.tensor_type)}")
            if len(preds[p].shape) > 2:
                raise ValueError(f"Too many dims in {p}")
        shapes = set([preds[p].shape for p in preds])
        if len(shapes) > 1: #TODO extend to force only first dim to be the same
            raise ValueError(f"shape mismatch. shapes of preds must be same")
        

In [ ]:
class Churn(ChurnMetric):
    """
    Simple Churn. Calculates number of classification disagreements, along axis:0. 
    Will take argmax if multiple columns.
    """
    def __init__(self, tensor_type="numpy", output_mode="proportion") -> None:
        super().__init__(tensor_type=tensor_type)
        if output_mode not in {"proportion", "count"}:
            raise ValueError("Unknown output_mode")
        self.output_mode = output_mode
    
    def __call__(self, predA: Union[np.ndarray, torch.Tensor], predB:Union[np.ndarray, torch.Tensor]) -> None:
        self.call_sanitize_inputs(predA=predA, predB=predB)
        
        if len(predA.shape) > 1:
            predA = predA.argmax(1)
            predB = predB.argmax(1)
        
        churn = sum(predA!=predB)

        if self.output_mode == "proportion":
            return churn / predA.shape[0]
        if self.output_mode == "count":
            return churn


In [ ]:
class WinLossRatio(ChurnMetric):
    """
    Lateral Churns are not loss
    """
    def __init__(self, tensor_type="numpy") -> None:
        super().__init__(tensor_type)
    
    def __call__(self, true_labels, pred_teacher, pred_student):
        self.call_sanitize_inputs(true_labels=true_labels, pred_teacher=pred_teacher, pred_student=pred_student)

        if len(pred_teacher.shape) > 1:
            pred_teacher = pred_teacher.argmax(1)
            pred_student = pred_student.argmax(1)
            true_labels = true_labels.argmax(1)

        pred_teacher = pred_teacher == true_labels
        pred_student = pred_student == true_labels
        wins = sum(pred_student > pred_teacher)
        losses = sum(pred_student < pred_teacher)

        return wins , losses


In [ ]:

class ChurnRatio(ChurnMetric):
    def __init__(self, tensor_type="numpy") -> None:
        super().__init__(tensor_type)
    
    def __call__(self, pred_teacher, pred_student, pred_control):
        self.call_sanitize_inputs(pred_teacher, pred_student, pred_control)
        
        pred_teacher = pred_teacher.argmax(1)
        pred_student = pred_student.argmax(1)
        pred_control = pred_control.argmax(1)

        churnratio = sum(pred_student!=pred_teacher) / sum(pred_control!=pred_teacher)
        return churnratio



In [ ]:
class GoodBadChurn(ChurnMetric):
    """
    lateral churn is bad
    """
    def __init__(self, tensor_type="numpy", mode=None, output_mode="proportion") -> None:
        super().__init__(tensor_type)
        if mode is None or mode not in {"good", "bad"}:
            raise ValueError("Please specify mode as good or bad")
        self.mode = mode
        if output_mode not in {"proportion", "count"}:
            raise ValueError("Unknown output_mode")
        self.output_mode = output_mode

    def __call__(self, true_labels, pred_teacher, pred_student):
        self.call_sanitize_inputs(true_labels=true_labels, pred_teacher=pred_teacher, pred_student=pred_student)
        
        pred_teacher = pred_teacher.argmax(1)
        pred_student = pred_student.argmax(1)
        true_labels = true_labels.argmax(1)

        
        if self.mode == "good":
            pred_teacher = pred_teacher == true_labels
            pred_student = pred_student == true_labels    
            churn = sum(pred_student > pred_teacher)
        elif self.mode == "bad":
            churn = sum((pred_student < pred_teacher))

        if self.output_mode == "proportion":
            return churn / pred_teacher.shape[0]
        if self.output_mode == "count":
            return churn

### Label Modification Methods

In [ ]:
#y_true when fed outside as argument to call should be one_hot encoded
class ChurnMethod():
    """
    Place Holder class that is other label modification methodology classes shold inherit from.
    Does nothing right now but can be modified to bring some functionality to all methods at once
    """
    def __init__(self):
        pass

class Distillation(ChurnMethod):
    def __init__(self, teacher, lamda=0.5):
        super().__init__()
        self.lamda = lamda
        self.teacher = teacher

    def __call__(self, X, y_true): #call an instance of the class to run this function. y_true needs to be one_hot encoded in this implementation
        with torch.no_grad(): #To ensure that pytorch is not creating gradients on the teacher model. i.e, all learning happens in student and double-ensuring teacher is static.
            teacher_pred = self.teacher(X) # The last layer of teacher model is just 10 neurons, all of which can take any real value
            teacher_label = nn.functional.softmax(self.teacher(X), dim=1) #convert output of model to softmax, getting probabilities for each class
            new_label = teacher_label * self.lamda + y_true * (1 - self.lamda) #distillation equation
            return new_label

class Anchor_RCP(ChurnMethod):
    def __init__(self, teacher, alpha=0.5, epsilon=0.5): #two hyperparams of anchor
        super().__init__()
        self.alpha = alpha
        self.epsilon = epsilon
        self.teacher = teacher

    def __call__(self, X, y_true):
        with torch.no_grad(): 
            teacher_pred = self.teacher(X)
            teacher_label = nn.functional.softmax(self.teacher(X), dim=1)
            correct_indices = y_true.argmax(1) == teacher_label.argmax(1) #see which indices are correct pred
            device = X.get_device() #in torch, all operations between models and variables can only be done when they are on the same device (cpu, gpu). here we get device of X. In general if you see something.cuda() being done somewhere, that sends the object to gpu
            new_label = torch.zeros(y_true.shape).to(device) #new variables are by default created on cpu. this line will transfer the new_label to whichever device X was on
            new_label[~correct_indices,:] = self.epsilon * y_true[~correct_indices,:] #all incorrect labels and are given the epsilon treatment from the rcp equation
            new_label[correct_indices,:] = teacher_label[correct_indices,:] * self.alpha + y_true[correct_indices,:] * (1 - self.alpha) #correct preds are given the alpha treatment from the rcp equation
            return new_label
        


In [ ]:
size, padding = 32, 4
transform = transforms.Compose(
    [
        transforms.RandomCrop(size, padding), transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

trainvalset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

split1 = torch.utils.data.random_split(trainvalset, [40000, 10000])
split2 = torch.utils.data.random_split(split1[0], [30000, 10000])
newtrainset = split1[0]
valset = split1[1]
oldtrainset = split2[0]

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
from torch.optim import lr_scheduler
old_train_loader = torch.utils.data.DataLoader(
        oldtrainset,
        batch_size=128,
        shuffle=True,
        num_workers=4,
        pin_memory=True
)
new_train_loader = torch.utils.data.DataLoader(
        newtrainset,
        batch_size=128,
        shuffle=True,
        num_workers=4,
        pin_memory=True
)
val_loader = torch.utils.data.DataLoader(
        valset,
        batch_size=128,
        shuffle=True,
        num_workers=4,
        pin_memory=True
)

# batchsize = 128
# SGD lr=0.1
# lr_scheduler
# random_crop, flipping, rotating



In [ ]:
class newmodel(nn.Module):
    def __init__(self, resnet) -> None:
        super().__init__()
        self.resnet = resnet
        self.final_layer = nn.Linear(1000, 10)

    def forward(self, x):
        x = nn.functional.relu(self.resnet(x))
        x = self.final_layer(x)
        return x

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target):
    """Computes the precision@k for the specified values of k"""
    maxk = 1
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in [1]:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res[0]





    #handle the learning rate scheduler.

# print(simple_churn(target_var, output.argmax(1)))


In [ ]:

mlflow.set_experiment('New run 2')

2022/11/14 04:32:01 INFO mlflow.tracking.fluent: Experiment with name 'New run 2' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/drive/MyDrive/Proofpoint/mlruns/2', creation_time=1668400321332, experiment_id='2', last_update_time=1668400321332, lifecycle_stage='active', name='New run 2', tags={}>

In [ ]:
def baseline_chunk():
    DECAY_EPOCHS= [20,120]
    DECAY= 0.1
    num_epochs = 30
    best_acc=0
    current_learning_rate = 0.1
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    baseline_model = newmodel(torch.hub.load('pytorch/vision:v0.13.1', 'resnet18', pretrained=False))
    criterion = nn.CrossEntropyLoss()
    baseline_optimizer = torch.optim.SGD(baseline_model.parameters(), lr=0.1, momentum=0.9)

    baseline_model.cuda()
    
    for epoch in range(0,num_epochs):
        if epoch in DECAY_EPOCHS and epoch != 0:
            current_learning_rate = current_learning_rate * DECAY
            for param_group in baseline_optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            print("Current learning rate has decayed to %f" %current_learning_rate)
        per_epoch_losses = []
        per_epoch_accuracies = []
        print(epoch)
        for j, (input, target) in enumerate(new_train_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target
            
            # compute output
            output = baseline_model(input_var)            
            loss = criterion(output, target_var)

            # compute gradient and do SGD step
            baseline_optimizer.zero_grad()
            loss.backward()
            baseline_optimizer.step()

            output = output.float()
            loss = loss.float()
            
            # measure accuracy and record loss
            per_epoch_accuracies.append(accuracy(output.data, target))
            per_epoch_losses.append(loss.item())

        per_epoch_val_losses = []
        per_epoch_val_accuracies = []
        for j, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target
            output = baseline_model(input_var)
            loss = criterion(output, target_var)
            output = output.float()
            loss = loss.float()
            
            # measure accuracy and record loss
            per_epoch_val_accuracies.append(accuracy(output.data, target))
            per_epoch_val_losses.append(loss.item())
      
        train_losses.append(sum(per_epoch_losses) / len(per_epoch_losses))
        val_losses.append(sum(per_epoch_val_losses) / len(per_epoch_val_losses))
        train_accuracies.append(sum(per_epoch_accuracies) / len(per_epoch_accuracies))
        val_accuracies.append(sum(per_epoch_val_accuracies) / len(per_epoch_val_accuracies))
      
        mlflow.log_metric("Baseline Train accuracy", train_accuracies[-1])
        mlflow.log_metric("Baseline Train loss", train_losses[-1])
        mlflow.log_metric("Baseline Val loss", val_losses[-1])
        mlflow.log_metric("Baseline Val accuracy", val_accuracies[-1])

        print("Training loss: %.4f, Training accuracy: %.4f, Val loss: %.4f, Val accuracy: %.4f, " %(train_losses[-1], train_accuracies[-1], val_losses[-1], val_accuracies[-1]))
        if val_accuracies[-1] > best_acc:
            best_acc = val_accuracies[-1]
            print('')

    print(f"=> Best: {best_acc:.4f}")
    return baseline_model



In [ ]:
def teacher_chunk():
    model = newmodel(torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)  
    model.cuda()
    DECAY_EPOCHS= [20,120]
    DECAY= 0.1
    num_epochs  = 30
    best_acc=0
    current_learning_rate = 0.1
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    for epoch in range(0,num_epochs):
        print("Epoch : %d" % epoch)
        if epoch in DECAY_EPOCHS and epoch != 0:
            current_learning_rate = current_learning_rate * DECAY
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            print("Current learning rate has decayed to %f" %current_learning_rate)
        per_epoch_losses = []
        per_epoch_accuracies = []
        for j, (input, target) in enumerate(old_train_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target
            
            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            output = output.float()
            loss = loss.float()
            
            # measure accuracy and record loss
            per_epoch_accuracies.append(accuracy(output.data, target))
            per_epoch_losses.append(loss.item())

        per_epoch_val_losses = []
        per_epoch_val_accuracies = []
        for j, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target
            output = model(input_var)
            loss = criterion(output, target_var)
            output = output.float()
            loss = loss.float()
            
            # measure accuracy and record loss
            per_epoch_val_accuracies.append(accuracy(output.data, target))
            per_epoch_val_losses.append(loss.item())
        
        train_losses.append(sum(per_epoch_losses) / len(per_epoch_losses))
        val_losses.append(sum(per_epoch_val_losses) / len(per_epoch_val_losses))
        train_accuracies.append(sum(per_epoch_accuracies) / len(per_epoch_accuracies))
        val_accuracies.append(sum(per_epoch_val_accuracies) / len(per_epoch_val_accuracies))
      
        mlflow.log_metric("Teacher Train accuracy", train_accuracies[-1])
        mlflow.log_metric("Teacher Train loss", train_losses[-1])
        mlflow.log_metric("Teacher Val loss", val_losses[-1])
        mlflow.log_metric("Teacher Val accuracy", val_accuracies[-1])

        print("Training loss: %.4f, Training accuracy: %.4f, Val loss: %.4f, Val accuracy: %.4f, " %(train_losses[-1], train_accuracies[-1], val_losses[-1], val_accuracies[-1]))
        if val_accuracies[-1] > best_acc:
            best_acc = val_accuracies[-1]
            print('')

    print(f"=> Best: {best_acc:.4f}")
    return model


In [ ]:
def student_chunk_rcp(model):
    DECAY_EPOCHS= [20,120]
    DECAY= 0.1
    num_epochs  = 30
    alphas = [0.2, 0.4, 0.6, 0.8]
    epsilon = 1.    

    student_models = {}
    for alpha in alphas:

      best_acc=0
      current_learning_rate = 0.1
      train_losses = []
      val_losses = []
      train_accuracies = []
      val_accuracies = []
      label_modifier = Anchor_RCP(model, alpha=alpha, epsilon=epsilon)
        ### END OF CHANGES ####
      print(f"alpha = {alpha}")
      print(f"epsilon = {epsilon}")
      student_model = newmodel(torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False))
      student_models[(alpha, epsilon)] = student_model
      criterion = nn.CrossEntropyLoss()
      student_optimizer = torch.optim.SGD(student_model.parameters(), lr=0.1, momentum=0.9)

      student_model.cuda()
      model.eval()

      for epoch in range(0,num_epochs):
          print("Epoch : %d" % epoch)
          if epoch in DECAY_EPOCHS and epoch != 0:
              current_learning_rate = current_learning_rate * DECAY
              for param_group in student_optimizer.param_groups:
                  param_group['lr'] = current_learning_rate
              print("Current learning rate has decayed to %f" %current_learning_rate)
          per_epoch_losses = []
          per_epoch_accuracies = []
          for j, (input, target) in enumerate(old_train_loader):
              target = target.cuda()
              input_var = input.cuda()
              target_var = nn.functional.one_hot(target, num_classes= 10)
              
              # compute output
              output = student_model(input_var)
              new_label = label_modifier(input_var, target_var)
              loss = criterion(output, new_label)
              student_optimizer.zero_grad()
              loss.backward()
              student_optimizer.step()
              output = output.float()
              loss = loss.float()
              
              # measure accuracy and record loss
              per_epoch_accuracies.append(accuracy(output.data, target))
              per_epoch_losses.append(loss.item())

          per_epoch_val_losses = []
          per_epoch_val_accuracies = []
          for j, (input, target) in enumerate(val_loader):
              target = target.cuda()
              input_var = input.cuda()
              target_var = target
              output = student_model(input_var)
              loss = criterion(output, target_var)
              output = output.float()
              loss = loss.float()
              
              # measure accuracy and record loss
              per_epoch_val_accuracies.append(accuracy(output.data, target))
              per_epoch_val_losses.append(loss.item())
          
          train_losses.append(sum(per_epoch_losses) / len(per_epoch_losses))
          val_losses.append(sum(per_epoch_val_losses) / len(per_epoch_val_losses))
          train_accuracies.append(sum(per_epoch_accuracies) / len(per_epoch_accuracies))
          val_accuracies.append(sum(per_epoch_val_accuracies) / len(per_epoch_val_accuracies))
        
          mlflow.log_metric(f"Student RCP Train accuracy_{alpha}", train_accuracies[-1])
          mlflow.log_metric(f"Student RCP  Train loss_{alpha}", train_losses[-1])
          mlflow.log_metric(f"Student RCP  Val loss_{alpha}", val_losses[-1])
          mlflow.log_metric(f"Student RCP  Val accuracy_{alpha}", val_accuracies[-1])
          

          print("Training loss: %.4f, Training accuracy: %.4f, Val loss: %.4f, Val accuracy: %.4f, " %(train_losses[-1], train_accuracies[-1], val_losses[-1], val_accuracies[-1]))
          if val_accuracies[-1] > best_acc:
              best_acc = val_accuracies[-1]
              print('')

      print(f"=> Best: {best_acc:.4f}")
      torch.save(student_model.state_dict(), f"alpha={alpha}_epsilon_{epsilon}.model")
    return student_models

In [ ]:
def student_chunk_dist(model):
    DECAY_EPOCHS= [20,120]
    DECAY= 0.1
    num_epochs  = 30
    
    lamdas = [0.2, 0.4, 0.6, 0.8]
    student_models = {}
    for lamda in lamdas:

      best_acc=0
      current_learning_rate = 0.1
      train_losses = []
      val_losses = []
      train_accuracies = []
      val_accuracies = []
      label_modifier = Distillation(model,lamda=lamda)
      print(f"lamda = {lamda}")
      student_model = newmodel(torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False))
      student_models[lamda] = student_model
      criterion = nn.CrossEntropyLoss()
      student_optimizer = torch.optim.SGD(student_model.parameters(), lr=0.1, momentum=0.9)

      student_model.cuda()
      model.eval()


      for epoch in range(0,num_epochs):
          print("Epoch : %d" % epoch)
          if epoch in DECAY_EPOCHS and epoch != 0:
              current_learning_rate = current_learning_rate * DECAY
              for param_group in student_optimizer.param_groups:
                  param_group['lr'] = current_learning_rate
              print("Current learning rate has decayed to %f" %current_learning_rate)
          per_epoch_losses = []
          per_epoch_accuracies = []
          for j, (input, target) in enumerate(new_train_loader):
              target = target.cuda()
              input_var = input.cuda()
              target_var = nn.functional.one_hot(target)

              # compute output
              output = student_model(input_var)
              teacher_label = nn.functional.softmax(model(input_var), dim=1)
              new_label = label_modifier(input_var, target_var)
              loss = criterion(output, new_label)

              # compute gradient and do SGD step
              student_optimizer.zero_grad()
              loss.backward()
              student_optimizer.step()

              output = output.float()
              loss = loss.float()
              
              # measure accuracy and record loss
              per_epoch_accuracies.append(accuracy(output.data, target))
              per_epoch_losses.append(loss.item())

          per_epoch_val_losses = []
          per_epoch_val_accuracies = []
          for j, (input, target) in enumerate(val_loader):
              target = target.cuda()
              input_var = input.cuda()
              target_var = target
              output = student_model(input_var)
              loss = criterion(output, target_var)
              output = output.float()
              loss = loss.float()
              
              # measure accuracy and record loss
              per_epoch_val_accuracies.append(accuracy(output.data, target))
              per_epoch_val_losses.append(loss.item())
          
          train_losses.append(sum(per_epoch_losses) / len(per_epoch_losses))
          val_losses.append(sum(per_epoch_val_losses) / len(per_epoch_val_losses))
          train_accuracies.append(sum(per_epoch_accuracies) / len(per_epoch_accuracies))
          val_accuracies.append(sum(per_epoch_val_accuracies) / len(per_epoch_val_accuracies))
        
          mlflow.log_metric(f"Student Distill Train accuracy_{lamda}", train_accuracies[-1])
          mlflow.log_metric(f"Student Distill Train loss_{lamda}", train_losses[-1])
          mlflow.log_metric(f"Student Distill Val loss_{lamda}", val_losses[-1])
          mlflow.log_metric(f"Student Distill Val accuracy_{lamda}", val_accuracies[-1])

          print("Training loss: %.4f, Training accuracy: %.4f, Val loss: %.4f, Val accuracy: %.4f, " %(train_losses[-1], train_accuracies[-1], val_losses[-1], val_accuracies[-1]))
          if val_accuracies[-1] > best_acc:
              best_acc = val_accuracies[-1]
              print('')

      print(f"=> Best: {best_acc:.4f}")
      torch.save(student_model.state_dict(), f"lambda{lamda}.modeel")
    return student_models

In [ ]:
#torch.save(model.state_dict(), f"teacher.modeel")

In [ ]:
def metric_chunk_dist(model, baseline_model, run_no):
  simple_churn = Churn(tensor_type="torch", output_mode="count")
  good_churn = GoodBadChurn(tensor_type="torch", mode="good", output_mode="count")
  bad_churn = GoodBadChurn(tensor_type="torch", mode="bad", output_mode="count")
  wlr = WinLossRatio(tensor_type="torch")

  baseline_wins = 0
  baseline_losses = 0
  baseline_good_churn = 0
  baseline_bad_churn = 0
  baseline_model.eval()
  churn_count_baseline = 0
  baseline_acc = 0
  teacher_acc = 0

  for i, (input, target) in enumerate(val_loader):
      input_var = input.cuda()
      target_var = target.cuda()
      target_var_oh = nn.functional.one_hot(target_var, num_classes=10)
      churn_count_baseline += simple_churn(model(input_var), baseline_model(input_var))
      baseline_acc += torch.sum(baseline_model(input_var).argmax(1) == target_var)
      teacher_acc += torch.sum(model(input_var).argmax(1) == target_var)
      baseline_win, baseline_loss = wlr(target_var_oh, model(input_var), baseline_model(input_var))
      baseline_wins += baseline_win
      baseline_losses += baseline_loss
      baseline_good_churn = good_churn(target_var_oh, model(input_var), baseline_model(input_var))
      baseline_bad_churn = bad_churn(target_var_oh, model(input_var), baseline_model(input_var))

  lamdas = [0.2, 0.4, 0.6, 0.8]

  for lamda in lamdas:
      student_model = newmodel(torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False))
      student_model.load_state_dict(torch.load(f"lambda{lamda}.modeel"))
      student_model.cuda()
      mlflow.pytorch.log_model(student_model, f"student_model_Val_{lamda}")
      
      simple_churn = Churn(tensor_type="torch", output_mode="count")
      good_churn = GoodBadChurn(tensor_type="torch", mode="good", output_mode="count")
      bad_churn = GoodBadChurn(tensor_type="torch", mode="bad", output_mode="count")
      wlr = WinLossRatio(tensor_type="torch")

      churn_count_student = 0
      student_acc = 0
      student_model.eval()
      student_wins = 0
      student_losses = 0
      student_good_churn = 0
      student_bad_churn = 0



      for i, (input, target) in enumerate(val_loader):
          input_var = input.cuda()
          target_var = target.cuda()
          target_var_oh = nn.functional.one_hot(target_var, num_classes=10)
          churn_count_student += simple_churn(model(input_var), student_model(input_var))        
          student_acc += torch.sum(student_model(input_var).argmax(1) == target_var)        
          student_win, student_loss = wlr(target_var_oh, model(input_var), student_model(input_var))        
          student_wins += student_win        
          student_losses += student_loss        
          student_good_churn += good_churn(target_var_oh, model(input_var), student_model(input_var))
          student_bad_churn = bad_churn(target_var_oh, model(input_var), student_model(input_var))
          

      churn_student = churn_count_student.item() / len(valset)
      churn_baseline = churn_count_baseline.item() / len(valset)
      print(f"\n\nlamda = {lamda}\n", )
      print(f"student churn = {churn_student}", )
      print(f"baseline churn = {churn_baseline}")
      print(f"student accuracy = {student_acc.item() / len(valset)}")
      print(f"baseline accuracy = {baseline_acc.item() / len(valset)}")
      print(f"teacher accuracy = {teacher_acc.item() / len(valset)}")
      print(f"student wlr = {(student_wins / student_losses)}")
      print(f"baseline wlr = {(baseline_wins / baseline_losses)}")
      print(f"churn ratio = {(churn_student/ churn_baseline)}")
      print(f"student good_churn = {student_good_churn.item() / len(valset)}")
      print(f"student bad_churn = {student_bad_churn.item() / len(valset)}")
      print(f"baseline good_churn = {baseline_good_churn.item() / len(valset)}")
      print(f"baseline bad_churn = {baseline_bad_churn.item() / len(valset)}")
      
      mlflow.log_metric(f"{run_no} student_churn_{lamda}", churn_student)
      mlflow.log_metric(f"{run_no} baseline churn_{lamda}", churn_baseline)
      mlflow.log_metric(f"{run_no} student accuracy_{lamda}", student_acc.item() / len(valset))
      mlflow.log_metric(f"{run_no} baseline accuracy_{lamda}", baseline_acc.item() / len(valset))
      mlflow.log_metric(f"{run_no} teacher accuracy _{lamda}", teacher_acc.item() / len(valset))
      mlflow.log_metric(f"{run_no} student wlr_{lamda}", (student_wins / student_losses))
      mlflow.log_metric(f"{run_no} baseline wlr_{lamda}", (baseline_wins / baseline_losses))
      mlflow.log_metric(f"{run_no} churn ratio_{lamda}", (churn_student/ churn_baseline))
      mlflow.log_metric(f"{run_no} student good_churn_{lamda}", student_good_churn.item() / len(valset))
      mlflow.log_metric(f"{run_no} student bad_churn _{lamda}", student_bad_churn.item() / len(valset))
      mlflow.log_metric(f"{run_no} baseline good_churn_{lamda}",baseline_good_churn.item() / len(valset))
      mlflow.log_metric(f"{run_no} baseline bad_churn_{lamda}", baseline_bad_churn.item() / len(valset))
      
      mlflow.pytorch.log_model(baseline_model, f"baseline_model_Val_{lamda}")




      

In [ ]:
def metric_chunk_rcp(model, baseline_model, run_no):
  simple_churn = Churn(tensor_type="torch", output_mode="count")
  good_churn = GoodBadChurn(tensor_type="torch", mode="good", output_mode="count")
  bad_churn = GoodBadChurn(tensor_type="torch", mode="bad", output_mode="count")
  wlr = WinLossRatio(tensor_type="torch")

  baseline_wins = 0
  baseline_losses = 0
  baseline_good_churn = 0
  baseline_bad_churn = 0
  baseline_model.eval()
  churn_count_baseline = 0
  baseline_acc = 0
  teacher_acc = 0

  for i, (input, target) in enumerate(val_loader):
      input_var = input.cuda()
      target_var = target.cuda()
      target_var_oh = nn.functional.one_hot(target_var, num_classes=10)
      churn_count_baseline += simple_churn(model(input_var), baseline_model(input_var))
      baseline_acc += torch.sum(baseline_model(input_var).argmax(1) == target_var)
      teacher_acc += torch.sum(model(input_var).argmax(1) == target_var)
      baseline_win, baseline_loss = wlr(target_var_oh, model(input_var), baseline_model(input_var))
      baseline_wins += baseline_win
      baseline_losses += baseline_loss
      baseline_good_churn = good_churn(target_var_oh, model(input_var), baseline_model(input_var))
      baseline_bad_churn = bad_churn(target_var_oh, model(input_var), baseline_model(input_var))

  alphas = [0.2, 0.4, 0.6, 0.8]
  epsilon = 1.


  for alpha in alphas:
        student_model = newmodel(torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False))
        student_model.load_state_dict(torch.load(f"alpha={alpha}_epsilon_{epsilon}.model"))
        student_model.cuda()
        mlflow.pytorch.log_model(student_model, f"Student_RCP_accuracy_alpha_{alpha}_epsilon_{epsilon}.model")
        
        simple_churn = Churn(tensor_type="torch", output_mode="count")
        good_churn = GoodBadChurn(tensor_type="torch", mode="good", output_mode="count")
        bad_churn = GoodBadChurn(tensor_type="torch", mode="bad", output_mode="count")
        wlr = WinLossRatio(tensor_type="torch")

        churn_count_student = 0
        student_acc = 0
        student_model.eval()
        student_wins = 0
        student_losses = 0
        student_good_churn = 0
        student_bad_churn = 0



        for i, (input, target) in enumerate(val_loader):
            input_var = input.cuda()
            target_var = target.cuda()
            target_var_oh = nn.functional.one_hot(target_var, num_classes=10)
            churn_count_student += simple_churn(model(input_var), student_model(input_var))        
            student_acc += torch.sum(student_model(input_var).argmax(1) == target_var)        
            student_win, student_loss = wlr(target_var_oh, model(input_var), student_model(input_var))        
            student_wins += student_win        
            student_losses += student_loss        
            student_good_churn += good_churn(target_var_oh, model(input_var), student_model(input_var))
            student_bad_churn = bad_churn(target_var_oh, model(input_var), student_model(input_var))
            

        churn_student = churn_count_student.item() / len(valset)
        churn_baseline = churn_count_baseline.item() / len(valset)
        print(f"\n\nalpha = {alpha}\n", )
        print(f"\n\nEpsilon = {epsilon}\n",)
        print(f"student churn = {churn_student}", )
        print(f"baseline churn = {churn_baseline}")
        print(f"student accuracy = {student_acc.item() / len(valset)}")
        print(f"baseline accuracy = {baseline_acc.item() / len(valset)}")
        print(f"teacher accuracy = {teacher_acc.item() / len(valset)}")
        print(f"student wlr = {(student_wins / student_losses)}")
        print(f"baseline wlr = {(baseline_wins / baseline_losses)}")
        print(f"churn ratio = {(churn_student/ churn_baseline)}")
        print(f"student good_churn = {student_good_churn.item() / len(valset)}")
        print(f"student bad_churn = {student_bad_churn.item() / len(valset)}")
        print(f"baseline good_churn = {baseline_good_churn.item() / len(valset)}")
        print(f"baseline bad_churn = {baseline_bad_churn.item() / len(valset)}")
        
        mlflow.log_metric(f"{run_no} student_churn_alpha_{alpha}_epsilon_{epsilon}", churn_student)
        mlflow.log_metric(f"{run_no} baseline churn_alpha_{alpha}_epsilon_{epsilon}", churn_baseline)
        mlflow.log_metric(f"{run_no} student accuracy_alpha_{alpha}_epsilon_{epsilon}", student_acc.item() / len(valset))
        mlflow.log_metric(f"{run_no} baseline accuracy_alpha_{alpha}_epsilon_{epsilon}", baseline_acc.item() / len(valset))
        mlflow.log_metric(f"{run_no} teacher accuracy _alpha_{alpha}_epsilon_{epsilon}", teacher_acc.item() / len(valset))
        mlflow.log_metric(f"{run_no} student wlr_alpha_{alpha}_epsilon_{epsilon}", (student_wins / student_losses))
        mlflow.log_metric(f"{run_no} baseline wlr_alpha_{alpha}_epsilon_{epsilon}", (baseline_wins / baseline_losses))
        mlflow.log_metric(f"{run_no} churn ratio_alpha_{alpha}_epsilon_{epsilon}", (churn_student/ churn_baseline))
        mlflow.log_metric(f"{run_no} student good_churn_{alpha}_epsilon_{epsilon}", student_good_churn.item() / len(valset))
        mlflow.log_metric(f"{run_no} student bad_churn _alpha_{alpha}_epsilon_{epsilon}", student_bad_churn.item() / len(valset))
        mlflow.log_metric(f"{run_no} baseline good_churn_alpha_{alpha}_epsilon_{epsilon}",baseline_good_churn.item() / len(valset))
        mlflow.log_metric(f"{run_no} baseline bad_churn_alpha_{alpha}_epsilon_{epsilon}", baseline_bad_churn.item() / len(valset))
        
        mlflow.pytorch.log_model(baseline_model, f"RCP baseline_model_Val_{alpha}_epsilon_{epsilon}")




      

Main loop for MLFlow:

In [ ]:
with mlflow.start_run(run_name="Ten Runs"):
    for run_no in range(10):
        teacher_model = teacher_chunk()
        rcp_models = student_chunk_rcp(teacher_model)
        dist_models = student_chunk_dist(teacher_model)
        base_model = baseline_chunk()
        metric_chunk_rcp(teacher_model, base_model, run_no)
        metric_chunk_dist(teacher_model, base_model, run_no)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch : 0
Training loss: 1.8454, Training accuracy: 31.6035, Val loss: 1.7637, Val accuracy: 35.7496, 

Epoch : 1
Training loss: 1.5903, Training accuracy: 42.0756, Val loss: 1.5129, Val accuracy: 45.1048, 

Epoch : 2
Training loss: 1.4101, Training accuracy: 49.0559, Val loss: 1.3630, Val accuracy: 51.8790, 

Epoch : 3
Training loss: 1.2824, Training accuracy: 54.6144, Val loss: 1.3046, Val accuracy: 55.7753, 

Epoch : 4
Training loss: 1.1684, Training accuracy: 58.8597, Val loss: 1.1646, Val accuracy: 58.8212, 

Epoch : 5
Training loss: 1.0897, Training accuracy: 61.5913, Val loss: 1.0963, Val accuracy: 61.0562, 

Epoch : 6
Training loss: 1.0214, Training accuracy: 64.0913, Val loss: 1.0717, Val accuracy: 63.0044, 

Epoch : 7
Training loss: 0.9648, Training accuracy: 65.9497, Val loss: 1.0343, Val accuracy: 64.3592, 

Epoch : 8
Training loss: 0.9235, Training accuracy: 67.6031, Val loss: 0.9567, Val accuracy: 66.7029, 

Epoch : 9
Training loss: 0.8678, Training accuracy: 70.0820, Val

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Training loss: 1.8422, Training accuracy: 32.1775, Val loss: 1.6435, Val accuracy: 40.5360, 

Epoch : 1
Training loss: 1.6042, Training accuracy: 42.1509, Val loss: 1.5578, Val accuracy: 42.0589, 

Epoch : 2
Training loss: 1.4263, Training accuracy: 49.0304, Val loss: 1.3973, Val accuracy: 48.3287, 

Epoch : 3
Training loss: 1.3229, Training accuracy: 53.5040, Val loss: 1.2680, Val accuracy: 55.6665, 

Epoch : 4
Training loss: 1.1756, Training accuracy: 58.5539, Val loss: 1.1570, Val accuracy: 58.9893, 

Epoch : 5
Training loss: 1.0753, Training accuracy: 62.7050, Val loss: 1.1098, Val accuracy: 61.7188, 

Epoch : 6
Training loss: 1.0149, Training accuracy: 65.0620, Val loss: 1.0522, Val accuracy: 63.0340, 

Epoch : 7
Training loss: 0.9595, Training accuracy: 67.0900, Val loss: 0.9518, Val accuracy: 67.0985, 

Epoch : 8
Training loss: 0.9064, Training accuracy: 69.1245, Val loss: 0.9321, Val accuracy: 67.4842, 

Epoch : 9
Training loss: 0.8568, Training accuracy: 70.9242, Val loss: 0.9

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


...

Training loss: 1.8465, Training accuracy: 31.5869, Val loss: 1.6250, Val accuracy: 39.0922, 

Epoch : 1
Training loss: 1.5772, Training accuracy: 42.7460, Val loss: 1.5267, Val accuracy: 44.0368, 

Epoch : 2
Training loss: 1.4252, Training accuracy: 49.0193, Val loss: 1.3409, Val accuracy: 51.6218, 

Epoch : 3
Training loss: 1.2850, Training accuracy: 54.6554, Val loss: 1.2118, Val accuracy: 57.8224, 

Epoch : 4
Training loss: 1.1705, Training accuracy: 59.1090, Val loss: 1.1739, Val accuracy: 58.9893, 

Epoch : 5
Training loss: 1.0807, Training accuracy: 62.6939, Val loss: 1.1224, Val accuracy: 60.8584, 

Epoch : 6
Training loss: 1.0152, Training accuracy: 65.4311, Val loss: 0.9963, Val accuracy: 65.0119, 

Epoch : 7
Training loss: 0.9592, Training accuracy: 67.1664, Val loss: 0.9850, Val accuracy: 65.3184, 

Epoch : 8
Training loss: 0.9141, Training accuracy: 68.8198, Val loss: 0.9454, Val accuracy: 67.6523, 

Epoch : 9
Training loss: 0.8664, Training accuracy: 70.4610, Val loss: 0.9

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Training loss: 1.8247, Training accuracy: 32.8480, Val loss: 1.6441, Val accuracy: 37.8659, 

Epoch : 1
Training loss: 1.5329, Training accuracy: 44.3163, Val loss: 1.4330, Val accuracy: 48.5067, 

Epoch : 2
Training loss: 1.3530, Training accuracy: 51.6611, Val loss: 1.3213, Val accuracy: 52.3339, 

Epoch : 3
Training loss: 1.2226, Training accuracy: 57.6164, Val loss: 1.2473, Val accuracy: 56.5763, 

Epoch : 4
Training loss: 1.1340, Training accuracy: 61.1370, Val loss: 1.1322, Val accuracy: 60.7199, 

Epoch : 5
Training loss: 1.0448, Training accuracy: 64.4459, Val loss: 1.0518, Val accuracy: 62.7373, 

Epoch : 6
Training loss: 1.0075, Training accuracy: 65.8688, Val loss: 0.9995, Val accuracy: 65.2393, 

Epoch : 7
Training loss: 0.9447, Training accuracy: 68.2535, Val loss: 0.9638, Val accuracy: 66.1986, 

Epoch : 8
Training loss: 0.9430, Training accuracy: 68.3500, Val loss: 0.9444, Val accuracy: 67.0392, 

Epoch : 9
Training loss: 0.9062, Training accuracy: 69.8227, Val loss: 0.9

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Training loss: 1.8303, Training accuracy: 33.2702, Val loss: 1.6606, Val accuracy: 39.3097, 

Epoch : 1
Training loss: 1.5680, Training accuracy: 43.7677, Val loss: 1.5388, Val accuracy: 42.5534, 

Epoch : 2
Training loss: 1.4021, Training accuracy: 50.1496, Val loss: 1.3237, Val accuracy: 51.5131, 

Epoch : 3
Training loss: 1.2622, Training accuracy: 56.2555, Val loss: 1.2771, Val accuracy: 54.3414, 

Epoch : 4
Training loss: 1.1710, Training accuracy: 59.8969, Val loss: 1.1399, Val accuracy: 59.4937, 

Epoch : 5
Training loss: 1.0915, Training accuracy: 62.7914, Val loss: 1.0849, Val accuracy: 62.2231, 

Epoch : 6
Training loss: 1.0342, Training accuracy: 65.2083, Val loss: 1.0394, Val accuracy: 64.2306, 

Epoch : 7
Training loss: 0.9799, Training accuracy: 67.0512, Val loss: 0.9740, Val accuracy: 65.2097, 

Epoch : 8
Training loss: 0.9281, Training accuracy: 69.4060, Val loss: 0.9568, Val accuracy: 66.3074, 

Epoch : 9
Training loss: 0.8914, Training accuracy: 70.8976, Val loss: 0.9

In [ ]:
while True:
  a=torch.Tensor([1,2,3,4,5])
  a.cuda()
  a=a*2

In [ ]:
!pip3 install pyngrok

In [ ]:

from pyngrok import ngrok
#Terminate open tunnels if exist
ngrok.kill()

#Setting the authtoken (optional)
#Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2HBNx1D7YdjGMMU5LPQQLxcTHeP_2GMdqwVRUMHyfftBwD9WV"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

#Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)



In [ ]:
!mlflow ui

[2022-11-14 17:48:07 +0000] [58408] [INFO] Starting gunicorn 20.1.0
[2022-11-14 17:48:07 +0000] [58408] [INFO] Listening at: http://127.0.0.1:5000 (58408)
[2022-11-14 17:48:07 +0000] [58408] [INFO] Using worker: sync
[2022-11-14 17:48:07 +0000] [58411] [INFO] Booting worker with pid: 58411
